# Llama-2 Chat

This notebook shows how to augment Llama-2 `LLM`s with the `Llama2Chat` wrapper to support the [Llama-2 chat prompt format](https://huggingface.co/blog/llama2#how-to-prompt-llama-2). Several `LLM` implementations in LangChain can be used as interface to Llama-2 chat models. These include [HuggingFaceTextGenInference](https://python.langchain.com/docs/integrations/llms/huggingface_textgen_inference), [LlamaCpp](https://python.langchain.com/docs/use_cases/question_answering/how_to/local_retrieval_qa), [GPT4All](https://python.langchain.com/docs/integrations/llms/gpt4all), ..., to mention a few examples. 

`Llama2Chat` is a generic wrapper that implements `BaseChatModel` and can therefore be used in applications as [chat model](https://python.langchain.com/docs/modules/model_io/models/chat/). `Llama2Chat` converts a list of [chat messages](https://python.langchain.com/docs/modules/model_io/models/chat/#messages) into the [required chat prompt format](https://huggingface.co/blog/llama2#how-to-prompt-llama-2) and forwards the formatted prompt as `str` to the wrapped `LLM`.

In [2]:
from langchain.chains import LLMChain
from langchain.chat_models import Llama2Chat
from langchain.memory import ConversationBufferMemory

For the chat application examples below, we'll use the following chat `prompt_template`:

In [3]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage

template_messages = [
    SystemMessage(content="You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{text}"),
]
prompt_template = ChatPromptTemplate.from_messages(template_messages)

## Chat with Llama-2 via `HuggingFaceTextGenInference` LLM

A [HuggingFaceTextGenInference](https://python.langchain.com/docs/integrations/llms/huggingface_textgen_inference) LLM encapsulates access to a [text-generation-inference](https://github.com/huggingface/text-generation-inference) server. In the following example, the inference server serves a [meta-llama/Llama-2-13b-chat-hf](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf) model. It can be started locally with:

```bash
docker run \
  --rm \
  --gpus all \
  --ipc=host \
  -p 8080:80 \
  -v ~/.cache/huggingface/hub:/data \
  -e HF_API_TOKEN=${HF_API_TOKEN} \
  ghcr.io/huggingface/text-generation-inference:0.9 \
  --hostname 0.0.0.0 \
  --model-id meta-llama/Llama-2-13b-chat-hf \
  --quantize bitsandbytes \
  --num-shard 4
```

This works on a machine with 4 x RTX 3080ti cards, for example. Adjust the `--num_shard` value to the number of GPUs available. The `HF_API_TOKEN` environment variable holds the Hugging Face API token.

In [ ]:
# !pip3 install text-generation

Create a `HuggingFaceTextGenInference` instance that connects to the local inference server and wrap it into `Llama2Chat`.

In [4]:
from langchain.llms import HuggingFaceTextGenInference

llm = HuggingFaceTextGenInference(
    inference_server_url="http://127.0.0.1:8080/",
    max_new_tokens=512,
    top_k=50,
    temperature=0.1,
    repetition_penalty=1.03,
)

model = Llama2Chat(llm=llm)

Then you are ready to use the chat `model` together with `prompt_template` and conversation `memory` in an `LLMChain`.

In [5]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(llm=model, prompt=prompt_template, memory=memory)

In [6]:
print(chain.run(text="What can I see in Vienna? Propose a few locations. Names only, no details."))

 Certainly! Here are a few popular locations to consider visiting in Vienna:

1. Schönbrunn Palace
2. St. Stephen's Cathedral
3. Hofburg Palace
4. Belvedere Palace
5. Prater Park
6. Vienna State Opera
7. Albertina Museum
8. Museum of Natural History
9. Kunsthistorisches Museum
10. Ringstrasse


In [7]:
print(chain.run(text="Tell me more about #2."))

 Sure! St. Stephen's Cathedral (Stephansdom) is a stunning Gothic cathedral located in the heart of Vienna. It's one of the city's most recognizable landmarks and a must-visit attraction for anyone interested in history, architecture, and religion.

Here are some interesting facts about St. Stephen's Cathedral:

1. Construction began in 1137 and took over 80 years to complete.
2. The cathedral features a unique blend of Gothic and Romanesque styles.
3. The south tower is 136 meters tall and offers breathtaking views of the city.
4. The cathedral's interior is adorned with intricate stone carvings, colorful stained glass windows, and impressive frescoes.
5. St. Stephen's Cathedral is the seat of the Archdiocese of Vienna and hosts many important religious events throughout the year.
6. The cathedral's crypt houses the tombs of many famous Viennese figures, including Mozart's family and several Austrian monarchs.
7. The cathedral's bell, known as the Pummerin, weighs over 20 tons and is 

## Chat with Llama-2 via `LlamaCPP` LLM

For using a Llama-2 chat model with a [LlamaCPP](https://python.langchain.com/docs/integrations/llms/llamacpp) `LMM`, install the `llama-cpp-python` library using [these installation instructions](https://python.langchain.com/docs/integrations/llms/llamacpp#installation). The following example uses a quantized [llama-2-7b-chat.Q4_0.gguf](https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_0.gguf) model stored locally at `~/Models/llama-2-7b-chat.Q4_0.gguf`. 

After creating a `LlamaCpp` instance, the `llm` is again wrapped into `Llama2Chat`

In [8]:
from os.path import expanduser
from langchain.llms import LlamaCpp

model_path = expanduser("~/Models/llama-2-7b-chat.Q4_0.gguf")

llm = LlamaCpp(model_path=model_path, streaming=False,)
model = Llama2Chat(llm=llm)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /home/martin/Models/llama-2-7b-chat.Q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_0     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_0     [  4096,  40

and used in the same way as in the previous example.

In [9]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(llm=model, prompt=prompt_template, memory=memory)

In [10]:
print(chain.run(text="What can I see in Vienna? Propose a few locations. Names only, no details."))

  Of course! Vienna is a beautiful city with a rich history and culture. Here are some must-see locations to consider:
1. Schönbrunn Palace
2. St. Stephen's Cathedral
3. Hofburg Palace
4. Belvedere Palace
5. MuseumsQuartier
6. Prater Park
7. Vienna State Opera
8. Burgtheater
9. Albertina Museum
10. Vienna Woods (Nussbaumer Forst)



llama_print_timings:        load time =   289.55 ms
llama_print_timings:      sample time =    40.63 ms /   104 runs   (    0.39 ms per token,  2559.50 tokens per second)
llama_print_timings: prompt eval time =  1587.24 ms /    48 tokens (   33.07 ms per token,    30.24 tokens per second)
llama_print_timings:        eval time =  6392.08 ms /   103 runs   (   62.06 ms per token,    16.11 tokens per second)
llama_print_timings:       total time =  8245.60 ms


In [11]:
print(chain.run(text="Tell me more about #2."))

Llama.generate: prefix-match hit


  Of course! St. Stephen's Cathedral, also known as Stephansdom, is a must-see attraction in Vienna. It is a beautiful Gothic church located in the heart of the city, on Stephansplatz. The cathedral was founded in the 12th century and has been a symbol of Vienna ever since.
Some of the notable features of St. Stephen's Cathedral include:
* Its impressive height, with a tower that reaches 137 meters (450 feet)
* The colorful tile roof, which is made up of more than 250,000 tiles and features intricate designs
* The stunning stained glass windows, which depict scenes from the Bible and Austrian history
* The marble altars and statues, including a famous statue of Mary Magdalene by sculptor Johann Georg von Hagen
* The fascinating history, including its role as a place of worship, a symbol of Vienna's power and influence, and a site of important historical events such as coronations and funerals.
Visitors can take a guided tour of the cathedral, climb to the top of the South Tower for



llama_print_timings:        load time =   289.55 ms
llama_print_timings:      sample time =    97.02 ms /   256 runs   (    0.38 ms per token,  2638.55 tokens per second)
llama_print_timings: prompt eval time =  3956.20 ms /   120 tokens (   32.97 ms per token,    30.33 tokens per second)
llama_print_timings:        eval time = 16141.25 ms /   256 runs   (   63.05 ms per token,    15.86 tokens per second)
llama_print_timings:       total time = 20754.33 ms
